In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import time
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

In [ ]:
data_2018= pd.read_csv("./league-2018-2024/league_2018.csv")
data_2019= pd.read_csv("./league-2018-2024/league_2019.csv")
data_2020= pd.read_csv("./league-2018-2024/league_2020.csv")
data_2021= pd.read_csv("./league-2018-2024/league_2021.csv")
data_2022= pd.read_csv("./league-2018-2024/league_2022.csv")
data_2023= pd.read_csv("./league-2018-2024/league_2023.csv")
data_2024= pd.read_csv("./league-2018-2024/league_2024.csv")
data = pd.concat([data_2018, data_2019, data_2020, data_2021, data_2022, data_2023, data_2024], ignore_index=True)
data.shape

/tmp/ipykernel_4007/1377208622.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2022= pd.read_csv("./league-2018-2024/league_2022.csv")
/tmp/ipykernel_4007/1377208622.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2023= pd.read_csv("./league-2018-2024/league_2023.csv")


(737376, 123)

In [ ]:
league_metrics = [
    "league", "year", "split","playoffs", "game", "patch", "ban1", "ban2", "ban3", "ban4", "ban5" ,
    'teamname',"gamelength", "kills", "deaths", "assists", 
    "firstblood", "team kpm", "ckpm", "dragons", "heralds", 
    "firstbaron", "barons", "firsttower", "towers", "inhibitors", "turretplates",
    "damagetochampions", "dpm", "damagetakenperminute", "wardsplaced",
    "wpm", "wardskilled", 
    "controlwardsbought", "visionscore", "totalgold", "earnedgold", "earned gpm", 
    "goldspent", "total cs", "cspm", "goldat10", "xpat10", "csat10", 
    "golddiffat10", "xpdiffat10", "csdiffat10", "killsat10", "assistsat10", 
    "deathsat10", "goldat15", "xpat15", "csat15",'result'
]

In [ ]:
lec_df = data[data['league'] == 'LEC']
lec_team_df = lec_df[league_metrics]
lec_team_df.reset_index(drop=True, inplace=True)
lec_team_df.head()

,league,year,split,playoffs,game,patch,ban1,ban2,ban3,ban4,...,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,goldat15,xpat15,csat15,result
0,LEC,2018,NaN,0,1.0,8.16,Gragas,Braum,Morgana,Ryze,...,295.0,-184.0,2.0,0.0,0.0,0.0,5651.0,7566.0,156.0,0
1,LEC,2018,NaN,0,1.0,8.16,Gragas,Braum,Morgana,Ryze,...,41.0,227.0,10.0,0.0,0.0,0.0,5734.0,7235.0,119.0,0
2,LEC,2018,NaN,0,1.0,8.16,Gragas,Braum,Morgana,Ryze,...,4.0,17.0,6.0,0.0,0.0,0.0,5584.0,8054.0,153.0,0
3,LEC,2018,NaN,0,1.0,8.16,Gragas,Braum,Morgana,Ryze,...,-613.0,-44.0,-1.0,0.0,0.0,0.0,5276.0,5544.0,128.0,0
4,LEC,2018,NaN,0,1.0,8.16,Gragas,Braum,Morgana,Ryze,...,-236.0,-519.0,-3.0,0.0,0.0,2.0,3420.0,3843.0,21.0,0


In [ ]:
player_columns_to_extract = [
    'playername', 'champion', 'kills', 'deaths', 'assists', 'damagetochampions',
    'dpm', 'damagetakenperminute', 'totalgold', 'earnedgold', 'earned gpm', 'goldspent',
    'total cs', 'cspm', 'goldat10', 'xpat10', 'csat10', 'golddiffat10', 'xpdiffat10',
    'csdiffat10', 'killsat10', 'assistsat10', 'deathsat10', 'goldat15', 'xpat15', 'csat15'
]

In [ ]:
player_data = {column: [] for column in player_columns_to_extract}
current_row = 0

while current_row < len(lec_df):
    # For each player in a team, extract the specified columns
    for column in player_columns_to_extract:
        column_data = lec_df.iloc[current_row:current_row + 10][column].tolist()
        player_data[column].extend(column_data)
    
    # Move to the next team and skip 2 rows (assuming teams are separated by 2 rows)
    current_row += 12

# Convert the dictionary to a DataFrame
players_df = pd.DataFrame(player_data)

players_df

,playername,champion,kills,deaths,assists,damagetochampions,dpm,damagetakenperminute,totalgold,earnedgold,...,csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,goldat15,xpat15,csat15
0,Alphari,Swain,2,2,4,31070.0,672.7535,685.9401,17633.0,11705.0,...,96.0,295.0,-184.0,2.0,0.0,0.0,0.0,5651.0,7566.0,156.0
1,Maxlore,Aatrox,1,0,7,16736.0,362.3818,809.5778,16829.0,10901.0,...,71.0,41.0,227.0,10.0,0.0,0.0,0.0,5734.0,7235.0,119.0
2,Sencux,Urgot,3,1,2,16085.0,348.2858,372.1039,17216.0,11288.0,...,98.0,4.0,17.0,6.0,0.0,0.0,0.0,5584.0,8054.0,153.0
3,Hans Sama,Kai'Sa,1,1,5,25963.0,562.1725,381.1332,18445.0,12517.0,...,82.0,-613.0,-44.0,-1.0,0.0,0.0,0.0,5276.0,5544.0,128.0
4,Mikyx,Alistar,1,8,4,4705.0,101.8766,485.9978,9593.0,3665.0,...,14.0,-236.0,-519.0,-3.0,0.0,0.0,2.0,3420.0,3843.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13355,Adam,Jax,2,5,1,10361.0,362.6954,1109.9300,10704.0,6932.0,...,77.0,-768.0,-506.0,-16.0,0.0,0.0,1.0,4834.0,6905.0,113.0
13356,Sheo,Brand,3,4,3,18941.0,663.0455,853.9323,9706.0,5934.0,...,81.0,422.0,451.0,21.0,1.0,0.0,0.0,5379.0,5581.0,110.0
13357,nuc,Azir,3,8,2,16363.0,572.8005,784.0957,11994.0,8222.0,...,94.0,-25.0,65.0,8.0,0.0,1.0,1.0,5658.0,7313.0,140.0
13358,Ice,Varus,2,3,3,14784.0,517.5263,410.1984,11068.0,7296.0,...,82.0,-193.0,-118.0,-5.0,0.0,0.0,0.0,5615.0,5512.0,131.0


In [ ]:
rows_to_delete = []

# Initialize variables to keep track of the current row
current_row = 0

while current_row < len(lec_team_df):
    # Identify the indices of rows to be deleted (10 consecutive rows)
    rows_to_delete.extend(list(range(current_row, current_row + 10)))
    
    # Move to the next set of rows to keep (2 consecutive rows)
    current_row += 12

# Drop the rows to be deleted from the original DataFrame
teams_df = lec_team_df.drop(rows_to_delete)

# Reset the index of the modified DataFrame
teams_df.reset_index(drop=True, inplace=True)
teams_df

,league,year,split,playoffs,game,patch,ban1,ban2,ban3,ban4,...,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,goldat15,xpat15,csat15,result
0,LEC,2018,NaN,0,1.0,8.16,Gragas,Braum,Morgana,Ryze,...,-509.0,-503.0,14.0,0.0,0.0,2.0,25665.0,32242.0,577.0,0
1,LEC,2018,NaN,0,1.0,8.16,Draven,Rakan,Kalista,Olaf,...,509.0,503.0,-14.0,2.0,4.0,0.0,23933.0,30973.0,557.0,1
2,LEC,2018,NaN,0,2.0,8.16,Gragas,Braum,Morgana,Ryze,...,-1299.0,-226.0,11.0,0.0,0.0,3.0,23612.0,30316.0,529.0,1
3,LEC,2018,NaN,0,2.0,8.16,Draven,Rakan,Kalista,Aatrox,...,1299.0,226.0,-11.0,3.0,4.0,0.0,23151.0,29198.0,509.0,0
4,LEC,2018,NaN,0,3.0,8.16,Gragas,Braum,Morgana,Poppy,...,442.0,371.0,-4.0,2.0,5.0,1.0,24166.0,28481.0,482.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,LEC,2024,Winter,1,1.0,14.02,Draven,Maokai,Brand,Braum,...,1259.0,856.0,14.0,3.0,7.0,0.0,24668.0,29896.0,555.0,0
2668,LEC,2024,Winter,1,2.0,14.02,Draven,LeBlanc,Twisted Fate,Hwei,...,-538.0,-282.0,-18.0,2.0,1.0,2.0,25398.0,29011.0,519.0,0
2669,LEC,2024,Winter,1,2.0,14.02,Ashe,Azir,Maokai,Vi,...,538.0,282.0,18.0,2.0,4.0,2.0,25051.0,29121.0,550.0,1
2670,LEC,2024,Winter,1,3.0,14.02,Ashe,Renata Glasc,Poppy,Darius,...,845.0,224.0,-9.0,2.0,2.0,1.0,28008.0,31483.0,519.0,1


In [ ]:
merged_df = pd.concat([teams_df, new_df], axis=1)

In [ ]:
merged_df.head()
merged_df.to_csv('lec.csv', index=False)

In [ ]:
from fastai.tabular.all import *

In [ ]:
df= pd.read_csv("./test.csv")

In [ ]:
df.shape

(58786, 41)

In [ ]:
categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()
continuous_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()


In [ ]:
print(continuous_features)

['league', 'year', 'split', 'playoffs', 'game', 'patch', 't1_side', 't2_side', 'teamname_1', 'teamname_2', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 't2_ban1', 't2_ban2', 't2_ban3', 't2_ban4', 't2_ban5', 'winning_team', 'playername_1', 'champion_1', 'playername_2', 'champion_2', 'playername_3', 'champion_3', 'playername_4', 'champion_4', 'playername_5', 'champion_5', 'playername_6', 'champion_6', 'playername_7', 'champion_7', 'playername_8', 'champion_8', 'playername_9', 'champion_9', 'playername_10', 'champion_10']


In [ ]:
dls = TabularDataLoaders.from_df (df, y_names="winning_team",
    cat_names = categorical_features,
    cont_names = continuous_features,)

In [ ]:
dls.show_batch()

,league,year,split,playoffs,game,patch,t1_side,t2_side,teamname_1,teamname_2,ban1,ban2,ban3,ban4,ban5,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5,winning_team,winning_team,playername_1,champion_1,playername_2,champion_2,playername_3,champion_3,playername_4,champion_4,playername_5,champion_5,playername_6,champion_6,playername_7,champion_7,playername_8,champion_8,playername_9,champion_9,playername_10,champion_10,winning_team,winning_team
0,4.0,2020.0,1.0,0.0,1.0,10.03,0.0,1.0,28.0,467.0,67.0,62.0,113.0,66.0,116.0,8.0,86.0,104.0,63.0,84.0,1.0,1.0,478.0,75.0,1768.0,94.0,3475.0,152.0,5446.0,141.0,6676.0,92.0,650.0,1.0,1753.0,36.0,3742.0,71.0,5316.0,74.0,6535.0,147.0,1.0,1.0
1,52.0,2022.0,8.0,1.0,2.0,12.14,0.0,1.0,1163.0,1185.0,30.0,118.0,140.0,45.0,84.0,17.0,26.0,50.0,89.0,35.0,1.0,1.0,1412.0,154.0,2913.0,88.0,4616.0,115.0,5970.0,24.0,7268.0,163.0,722.0,37.0,2662.0,124.0,3772.0,103.0,6043.0,102.0,2462.0,117.0,1.0,1.0
2,22.0,2022.0,1.0,1.0,2.0,12.05,0.0,1.0,943.0,421.0,152.0,163.0,38.0,92.0,4.0,125.0,62.0,150.0,46.0,33.0,0.0,0.0,472.0,98.0,1082.0,63.0,4211.0,99.0,5284.0,48.0,6885.0,104.0,837.0,57.0,2718.0,135.0,4277.0,132.0,5172.0,141.0,6775.0,64.0,0.0,0.0
3,36.0,2020.0,0.0,0.0,1.0,10.03,0.0,1.0,66.0,535.0,64.0,90.0,113.0,30.0,95.0,122.0,152.0,86.0,79.0,98.0,0.0,0.0,385.0,104.0,2463.0,44.0,3783.0,75.0,4956.0,74.0,6569.0,151.0,461.0,106.0,2422.0,96.0,3890.0,85.0,5310.0,8.0,3345.0,15.0,0.0,0.0
4,4.0,2019.0,8.0,0.0,2.0,9.11,0.0,1.0,324.0,464.0,49.0,80.0,147.0,79.0,46.0,41.0,50.0,1.0,92.0,76.0,0.0,0.0,23.0,57.0,1762.0,94.0,3777.0,99.0,5035.0,141.0,6608.0,69.0,22.0,98.0,2310.0,84.0,3893.0,115.0,5312.0,67.0,6721.0,15.0,0.0,0.0
5,75.0,2023.0,8.0,0.0,1.0,13.11,0.0,1.0,1060.0,1269.0,72.0,134.0,150.0,82.0,45.0,132.0,94.0,148.0,116.0,70.0,1.0,1.0,1639.0,158.0,3222.0,140.0,4659.0,142.0,5998.0,67.0,6819.0,159.0,1033.0,86.0,2736.0,135.0,3851.0,7.0,6118.0,8.0,6822.0,147.0,1.0,1.0
6,3.0,2018.0,8.0,1.0,1.0,8.15,0.0,1.0,16.0,23.0,76.0,117.0,49.0,124.0,19.0,92.0,4.0,9.0,36.0,17.0,1.0,1.0,17.0,33.0,1755.0,101.0,3464.0,1.0,4890.0,114.0,6298.0,106.0,24.0,35.0,1763.0,84.0,707.0,99.0,4896.0,130.0,6304.0,15.0,1.0,1.0
7,18.0,2021.0,1.0,0.0,1.0,11.05,0.0,1.0,136.0,690.0,85.0,65.0,123.0,131.0,49.0,103.0,122.0,38.0,46.0,17.0,0.0,0.0,167.0,78.0,2233.0,128.0,3576.0,98.0,5004.0,141.0,6404.0,4.0,174.0,35.0,1912.0,84.0,3946.0,99.0,5378.0,47.0,3887.0,162.0,0.0,0.0
8,34.0,2022.0,16.0,0.0,1.0,12.05,0.0,1.0,50.0,376.0,79.0,64.0,150.0,116.0,3.0,2.0,51.0,99.0,62.0,126.0,0.0,0.0,29.0,95.0,2876.0,63.0,3542.0,32.0,5684.0,67.0,6378.0,77.0,528.0,131.0,2659.0,38.0,4016.0,136.0,5441.0,48.0,5105.0,104.0,0.0,0.0
9,42.0,2021.0,0.0,0.0,4.0,11.21,0.0,1.0,864.0,295.0,147.0,126.0,24.0,41.0,154.0,62.0,37.0,16.0,46.0,57.0,0.0,0.0,443.0,104.0,1999.0,63.0,3763.0,116.0,4592.0,141.0,6248.0,64.0,1111.0,17.0,2221.0,88.0,3899.0,152.0,5609.0,67.0,6649.0,77.0,0.0,0.0


In [ ]:
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,accuracy,time
0,0.015833,0.009710,0.535256,00:06
1,0.005877,0.000750,0.535256,00:06
2,0.005456,0.000488,0.535256,00:06
3,0.004919,0.000540,0.535256,00:06
